# Import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import KFold,LeaveOneOut,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import xgboost as xgb
import matplotlib.pyplot as plt

import scipy.io
from glob import glob
import numpy as np
from tqdm import tqdm
import pandas as pd

# Load data

In [ ]:
IDD='/content/drive/MyDrive/Môn học ML/EEG_IDD_TDC/Data/CleanData/CleanData_IDD'
TDC='/content/drive/MyDrive/Môn học ML/EEG_IDD_TDC/Data/CleanData/CleanData_TDC'

In [ ]:
IDD_Music=[]
for i in glob(IDD+'/Music/*.mat'):
    data=scipy.io.loadmat(i)['clean_data'].reshape(14,-1,128*4)
    data=np.swapaxes(data,0,1)
    IDD_Music.append(data )

TDC_Music=[]
for i in glob(TDC+'/Music/*.mat'):
    data=scipy.io.loadmat(i)['clean_data'].reshape(14,-1,128*4)
    data=np.swapaxes(data,0,1)
    TDC_Music.append(data )

In [ ]:
IDD_Music

[array([[[ 1.76906530e-02,  2.24826889e+01,  3.23705063e+01, ...,
          -3.75554419e+00, -7.95021725e+00, -1.46302691e+01],
         [ 1.74275413e-02,  2.86000004e+01,  3.79585991e+01, ...,
           6.69905758e+00,  1.30137646e+00, -5.45933580e+00],
         [ 1.78194381e-02,  8.45523643e+00,  1.13207703e+01, ...,
          -1.72296963e+01, -1.32111425e+01, -1.06085424e+01],
         ...,
         [ 1.66808069e-02,  6.09420681e+00,  7.52591324e+00, ...,
          -1.07911377e+01, -7.44947100e+00, -4.66683435e+00],
         [ 1.71255060e-02,  1.60776100e+01,  2.46162834e+01, ...,
          -2.38822937e-01,  4.14311218e+00,  5.36195326e+00],
         [ 1.70411710e-02,  2.41787834e+01,  3.39462662e+01, ...,
          -5.98148108e+00, -4.32205009e+00, -3.23641372e+00]],
 
        [[-2.12174664e+01, -2.32392178e+01, -2.17232971e+01, ...,
           5.02078247e+00,  1.04973803e+01,  6.84556293e+00],
         [-9.54399776e+00, -8.45391369e+00, -5.36752987e+00, ...,
           5.67579889

In [ ]:
len(TDC_Music)

7

# Calcualte Features

In [ ]:
import pywt

import pywt

def wavelet_coefs(data, func, wavelet_name='db4'):
    coefs = pywt.wavedec(data, wavelet=wavelet_name, level=None)
    n_channels, n_times = data.shape
    levdec = len(coefs) - 1

    wavelet_stats = np.zeros((n_channels, levdec))
    for j in range(n_channels):
        for l in range(levdec):
            if func == 'energy':
                wavelet_stats[j, l] = np.sum(coefs[levdec - l][j, :] ** 2)
            elif func == 'std':
                wavelet_stats[j, l] = np.std(coefs[levdec - l][j, :])
            elif func == 'avg':
                wavelet_stats[j, l] = np.mean(coefs[levdec - l][j, :])

    return wavelet_stats.ravel()



'''
Hàm wavelet_coefs thực hiện các bước sau:

Phân giải tín hiệu đầu vào thành các hệ số wavelet bằng cách sử dụng wavelet Daubechies 4 (mặc định).
Tính toán năng lượng hoặc một đặc trưng khác từ các hệ số wavelet cho mỗi kênh và mỗi mức độ phân giải.
Trả về các đặc trưng này dưới dạng một mảng một chiều.
'''

'\nHàm wavelet_coefs thực hiện các bước sau:\n\nPhân giải tín hiệu đầu vào thành các hệ số wavelet bằng cách sử dụng wavelet Daubechies 4 (mặc định).\nTính toán năng lượng hoặc một đặc trưng khác từ các hệ số wavelet cho mỗi kênh và mỗi mức độ phân giải.\nTrả về các đặc trưng này dưới dạng một mảng một chiều.\n'

In [ ]:
def calculate_indivisual_features(func):
  features1=[]
  features2=[]
  for k in tqdm(range(7)):
    feature1=[]
    feature2=[]
    for i in range(30):
      feature1.append(wavelet_coefs(IDD_Music[k][i],func=func))
      feature2.append(wavelet_coefs(TDC_Music[k][i],func=func))
    features1.append(np.array(feature1))
    features2.append(np.array(feature2))
  x1=np.array(features1)
  x2=np.array(features2)
  y=np.concatenate(((np.zeros(x1.shape[0])),(np.ones(x2.shape[0]))))
  X=np.concatenate((x1,x2),axis=0)
  return X,y

In [ ]:
classifiers = [
    KNeighborsClassifier(),
    SVC(),
    NuSVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    #xgb.XGBClassifier(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]

In [ ]:
def classifiers_calc(X,y):
  accuracy_avg=[]
  accuracy_std=[]
  f1_avg=[]
  f1_std=[]
  for clfs in classifiers:
      # print('====================================')
      # name = clfs.__class__.__name__
      # print(name)
      acc_scores=[]
      f1_scores=[]

      for train,test in StratifiedKFold(7).split(X,y):

        X_train=X[train].reshape(-1,X.shape[2])
        X_test=X[test].reshape(-1,X.shape[2])

        y_train=np.concatenate([[i]*X.shape[1] for i in y[train]])
        y_test=np.concatenate([[i]*X.shape[1] for i in y[test]])

        sc=StandardScaler()

        X_train=sc.fit_transform(X_train)
        X_test=sc.transform(X_test)

        X_train, y_train = shuffle(X_train, y_train)

        clfs.fit(X_train,y_train)

        y_pred=clfs.predict(X_test)

        acc_scores.append(accuracy_score(y_test,y_pred))
        f1_scores.append(f1_score(y_test,y_pred))

      accuracy_avg.append(np.mean(acc_scores))
      accuracy_std.append(np.std(acc_scores))

      f1_avg.append(np.mean(f1_scores))
      f1_std.append(np.std(f1_scores))

  return accuracy_avg,accuracy_std,f1_avg,f1_std

In [ ]:
 classifier=['KNN','c-SVM','nu-SVM','DT','RF','AB','GB','NB','LDA','QDA','LR']
def plot_classifiers(accuracy_avg,f1_avg,title):

  y_pos = np.arange(len(classifier))
  w=0.4
  plt.figure(figsize=(10,5))
  plt.bar(y_pos,np.array(accuracy_avg),align='center',width=w,label='Accuracy')
  plt.bar(y_pos+w,np.array(f1_avg),align='center',width=w,label='F1-score')
  ya=np.array(accuracy_avg)
  for index, value in enumerate(ya):
      plt.text(index-0.1,value-0.2, str(np.round(value,2)),rotation=90,color='white',fontsize=11)

  yf=np.array(f1_avg)
  for index, value in enumerate(yf):
      plt.text(index+w-0.1,value-0.1, str(np.round(value,2)),rotation=90,color='white',fontsize=11)


  plt.xticks(y_pos, classifier,fontsize=11)
  plt.yticks(fontsize=11)

  plt.ylabel('Score',fontsize=12)
  plt.xlabel('Classifiers',fontsize=12)
  plt.title('Classifiers Performance of {} Coefficents Features'.format(title),fontsize=12)
  plt.legend()
  plt.savefig('{} Feature.eps'.format(title))

## Plot classifiers performance

## Average coefficents

In [ ]:
X,y= calculate_indivisual_features(func = 'avg')

100%|██████████| 7/7 [00:00<00:00, 12.98it/s]


In [ ]:
y

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
meanFeatureAcc,meanFeatureAccStd,meanFeatureF1,meanFeatureF1Std=classifiers_calc(X,y)

In [ ]:
len(meanFeatureAcc)

11

In [ ]:
df_1=pd.DataFrame(zip(meanFeatureAcc,meanFeatureAccStd,meanFeatureF1,meanFeatureF1Std)).round(4)*100
df_1.columns=['Acc Avg', 'Acc Std.','F1 Avg', 'F1 Std.']
mean_df_1=df_1.rename(index=dict(zip(list(range(0,11)),classifier)))
mean_df_1

,Acc Avg,Acc Std.,F1 Avg,F1 Std.
KNN,70.95,10.91,78.03,6.77
c-SVM,80.00,15.53,80.37,15.48
nu-SVM,81.43,14.18,82.09,13.90
DT,72.62,9.87,71.96,13.45
RF,81.43,12.64,81.58,13.60
AB,57.38,11.12,58.37,12.25
GB,87.86,7.54,88.96,6.24
NB,55.95,18.25,56.93,20.27
LDA,56.19,4.15,58.48,5.46
QDA,91.90,8.14,90.28,10.34


### Energy


In [ ]:
X_2,y_2=calculate_indivisual_features(func='energy')
accuracy_avg_2,    accuracy_std_2,   f1_avg_2,   f1_std_2  =  classifiers_calc(X_2,y_2)
df_2=pd.DataFrame(zip(accuracy_avg_2,    accuracy_std_2,   f1_avg_2,   f1_std_2),classifier).round(4)*100
df_2.columns=['Acc Avg', 'Acc Std.','F1 Avg', 'F1 Std.']
df_2

100%|██████████| 7/7 [00:00<00:00, 10.95it/s]


,Acc Avg,Acc Std.,F1 Avg,F1 Std.
KNN,68.81,19.77,75.96,13.98
c-SVM,65.24,22.03,66.51,24.84
nu-SVM,61.67,22.27,62.56,26.82
DT,56.90,26.06,62.62,28.18
RF,68.57,24.81,71.39,28.27
AB,65.48,25.27,72.12,23.50
GB,65.71,22.87,71.16,21.70
NB,51.43,24.73,43.27,26.85
LDA,65.71,17.16,73.56,10.39
QDA,63.81,24.64,40.53,40.75


std

In [ ]:
X_3,y_3=calculate_indivisual_features(func='std')
accuracy_avg_3,    accuracy_std_3,   f1_avg_3,   f1_std_3  =  classifiers_calc(X_3,y_3)
df_3=pd.DataFrame(zip(accuracy_avg_3,    accuracy_std_3,   f1_avg_3,   f1_std_3),classifier).round(4)*100
df_3.columns=['Acc Avg', 'Acc Std.','F1 Avg', 'F1 Std.']
df_3

100%|██████████| 7/7 [00:03<00:00,  2.05it/s]


,Acc Avg,Acc Std.,F1 Avg,F1 Std.
KNN,64.76,21.77,73.73,14.93
c-SVM,66.90,21.97,69.96,22.46
nu-SVM,64.05,22.95,65.73,25.66
DT,56.43,26.25,64.75,23.80
RF,71.19,24.51,73.04,26.91
AB,62.86,12.17,62.42,24.22
GB,65.95,23.14,71.55,22.00
NB,57.62,23.50,52.78,26.33
LDA,64.52,15.73,69.80,13.91
QDA,55.71,18.06,27.00,31.79


combination

In [ ]:
def combine_wavelet_features(data, wavelet_name='db4'):
    # Calculate each feature using the wavelet_coefs function
    energy_features = wavelet_coefs(data, 'energy', wavelet_name)
    std_features = wavelet_coefs(data, 'std', wavelet_name)
    avg_features = wavelet_coefs(data, 'avg', wavelet_name)

    # Concatenate all features into a single feature vector
    combined_features = np.concatenate((energy_features, std_features, avg_features))

    return combined_features

In [ ]:
def calculate_combination_features():
  features1=[]
  features2=[]
  for k in tqdm(range(7)):
    feature1=[]
    feature2=[]
    for i in range(30):
      feature1.append(combine_wavelet_features(IDD_Music[k][i]))
      feature2.append(combine_wavelet_features(TDC_Music[k][i]))
    features1.append(np.array(feature1))
    features2.append(np.array(feature2))
  x1=np.array(features1)
  x2=np.array(features2)
  y=np.concatenate(((np.zeros(x1.shape[0])),(np.ones(x2.shape[0]))))
  X=np.concatenate((x1,x2),axis=0)
  return X,y

In [ ]:
X_4,y_4= calculate_combination_features()
accuracy_avg_4,    accuracy_std_4,   f1_avg_4,   f1_std_4  =  classifiers_calc(X_4,y_4)
df_2=pd.DataFrame(zip(accuracy_avg_4,    accuracy_std_4,   f1_avg_4,   f1_std_4),classifier).round(4)*100
df_2.columns=['Acc Avg', 'Acc Std.','F1 Avg', 'F1 Std.']
df_2

100%|██████████| 7/7 [00:03<00:00,  1.89it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/l

,Acc Avg,Acc Std.,F1 Avg,F1 Std.
KNN,67.62,18.06,75.49,12.01
c-SVM,67.86,21.43,67.95,24.10
nu-SVM,65.95,24.18,65.14,28.23
DT,54.76,27.54,64.21,23.69
RF,66.19,24.38,69.60,27.68
AB,67.14,24.92,71.81,26.46
GB,71.19,22.62,74.62,20.96
NB,55.48,23.68,50.01,26.62
LDA,79.29,14.80,83.29,10.06
QDA,53.33,11.62,43.71,26.32
